In [43]:
from utils.util import load_json_line_by_line
from collections import Counter
data = load_json_line_by_line("./output\ECIhardprompt\generated_predictions (7).jsonl")
def get_label_from_str(text):
    return text.split("\n", 1)[0]    
def find_match_substring(text):
    if "none" in text:
        return 0
    elif "caused by" in text:
        return -1
    elif "cause" in text:
        return 1
    else:
        return 2
#label2index 1 -1 0
label2index = {"event1 and event2 have no causal relationship":0,"event1 caused by event2":-1,"event1 cause event2":1}
ground_truth = [label2index[d["label"]] for d in data]
predictions = [d["predict"] for d in data]
true_predictions = [get_label_from_str(p) for p in predictions]
predictions1 = [find_match_substring(p) for p in true_predictions]
print(Counter(predictions1))
print(Counter(ground_truth))
#get content before \n



Counter({0: 728, 1: 232, -1: 39, 2: 1})
Counter({0: 839, -1: 82, 1: 79})


In [44]:
from collections import defaultdict  
#错误，误用
def calculate_per_class_metrics(ground_truth, predictions):  
    # 创建字典来存储每个类别的TP, FP, FN计数  
    tp_fp_fn = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})  
      
    # 遍历所有样本  
    for gt, pred in zip(ground_truth, predictions):  
        # 如果预测正确，增加TP计数  
        if gt == pred:  
            tp_fp_fn[gt]['TP'] += 1  
        # 如果预测错误但预测为正例，增加FP计数  
        elif pred != -1 and gt != pred:  # 假设-1是未分类或填充值，根据实际情况调整  
            tp_fp_fn[pred]['FP'] += 1  
        # 如果预测错误且实际为正例但预测为负例或未分类，增加FN计数  
        elif gt != -1 and gt != pred:  
            tp_fp_fn[gt]['FN'] += 1  
      
    # 计算每个类别的精确度、召回率和F1分数  
    metrics_per_class = {}  
    for label, counts in tp_fp_fn.items():  
        tp = counts['TP']  
        fp = counts['FP']  
        fn = counts['FN']  
          
        # 避免除以零的情况  
        precision = tp / (tp + fp) if tp + fp > 0 else 0  
        recall = tp / (tp + fn) if tp + fn > 0 else 0  
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0  
          
        metrics_per_class[label] = {  
            'Precision': precision,  
            'Recall': recall,  
            'F1 Score': f1_score  
        }  

    # 返回整体的P、R和F1         
    return metrics_per_class  
def calculate_accuracy(ground_truth, predictions):  
    # 使用列表推导式计算正确分类的样本数  
    correct_predictions = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == pred)  
      
    # 计算准确率  
    accuracy = correct_predictions / len(ground_truth)  
      
    # 返回准确率  
    return accuracy    

metrics = calculate_per_class_metrics(ground_truth, predictions1)  
for label, metric_values in metrics.items():  
    print(f"Label {label}: Precision={metric_values['Precision']:.2f}, Recall={metric_values['Recall']:.2f}, F1 Score={metric_values['F1 Score']:.2f}")
print(calculate_accuracy(ground_truth,predictions1))
print(Counter(ground_truth))


Label 0: Precision=0.86, Recall=0.96, F1 Score=0.91
Label 1: Precision=0.11, Recall=0.84, F1 Score=0.20
Label -1: Precision=1.00, Recall=1.00, F1 Score=1.00
Label 2: Precision=0.00, Recall=0.00, F1 Score=0.00
0.658
Counter({0: 839, -1: 82, 1: 79})


In [45]:
from sklearn.metrics import precision_recall_fscore_support

def calculate_prf_per_class(predictions, ground_truth):
    # Calculate precision, recall, and F1-score for each class
    precision, recall, f1, support = precision_recall_fscore_support(ground_truth, predictions, average=None)
    oprecision, orecall, of1, osupport =precision_recall_fscore_support(ground_truth, predictions, average="weighted")
    # Create a dictionary to store results for each class
    class_results = {}
    for i in range(len(precision)):
        class_results[i] = {
            'precision': precision[i],
            'recall': recall[i],
            'f1-score': f1[i],
            'support': support[i]
        }
    class_results[len(precision)]={
            'precision': oprecision,
            'recall': orecall,
            'f1-score': of1,
            'support': osupport
        }
    return class_results

# Example usage:
# Replace these with your actual predictions and ground truth lists

class_results = calculate_prf_per_class(predictions1, ground_truth)

# Display results for each class
for class_label, metrics in class_results.items():
    print("Class:", class_label)
    print("Precision:", metrics['precision'])
    print("Recall:", metrics['recall'])
    print("F1-score:", metrics['f1-score'])
    print("Support:", metrics['support'])
    print()


Class: 0
Precision: 0.15384615384615385
Recall: 0.07317073170731707
F1-score: 0.09917355371900825
Support: 82

Class: 1
Precision: 0.8598901098901099
Recall: 0.7461263408820024
F1-score: 0.7989789406509252
Support: 839

Class: 2
Precision: 0.11206896551724138
Recall: 0.3291139240506329
F1-score: 0.1672025723472669
Support: 79

Class: 3
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Support: 0

Class: 4
Precision: 0.742916635089049
Recall: 0.658
F1-score: 0.6916845658265192
Support: None



D:\miniconda3\envs\llmprocedure\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\miniconda3\envs\llmprocedure\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
from utils.util import load_json_line_by_line
from collections import Counter
import re
data = load_json_line_by_line("./output\ECIhardprompt\generated_predictions (11).jsonl")
def get_label_from_str(text):
    return text.split(".", 1)[0]    
def extract_label(input_string):
    # Define a regular expression pattern to match the label substring
    pattern = r'<c>(.*?)</c>'
    
    # Use re.findall to find all matches of the pattern in the input string
    matches = re.findall(pattern, input_string)
    
    # Return the first match if found, or None if no match is found
    return matches[0] if matches else None
def find_match_substring(text):
    if "none"==text:
        return 0
    elif "caused by"==text:
        return -1
    elif "cause"==text:
        return 1
    else:
        return 2
#label2index 1 -1 0
label2index = {"event1 and event2 have no causal relationship":0,"event1 caused by event2":-1,"event1 cause event2":1}
ground_truth = [label2index[d["label"]] for d in data]
predictions = [d["predict"] for d in data]
true_predictions = [extract_label(p) for p in predictions]
predictions1 = [find_match_substring(p) for p in true_predictions]
print(Counter(predictions1))
print(Counter(ground_truth))
#get content before \n


Counter({1: 688, 0: 204, -1: 63, 2: 45})
Counter({0: 839, -1: 82, 1: 79})


In [48]:
class_results = calculate_prf_per_class(predictions1, ground_truth)
print(calculate_accuracy(ground_truth,predictions1))
# Display results for each class
for class_label, metrics in class_results.items():
    print("Class:", class_label)
    print("Precision:", metrics['precision'])
    print("Recall:", metrics['recall'])
    print("F1-score:", metrics['f1-score'])
    print("Support:", metrics['support'])
    print()

print(Counter(ground_truth))

0.252
Class: 0
Precision: 0.09523809523809523
Recall: 0.07317073170731707
F1-score: 0.08275862068965516
Support: 82

Class: 1
Precision: 0.8872549019607843
Recall: 0.21573301549463647
F1-score: 0.34707574304889743
Support: 839

Class: 2
Precision: 0.09447674418604651
Recall: 0.8227848101265823
F1-score: 0.16949152542372883
Support: 79

Class: 3
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Support: 0

Class: 4
Precision: 0.7596800493453195
Recall: 0.252
F1-score: 0.3113725858230513
Support: None

Counter({0: 839, -1: 82, 1: 79})


D:\miniconda3\envs\llmprocedure\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\miniconda3\envs\llmprocedure\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
import csv  
from collections import Counter  
  
# 假设 class_results 是已经计算好的字典，其结构如下：  
# class_results = {  
#     'class1': {'precision': 0.8, 'recall': 0.9, 'f1-score': 0.85, 'support': 100},  
#     'class2': {'precision': 0.7, 'recall': 0.8, 'f1-score': 0.75, 'support': 200},  
#     # ... 更多类别  
# }  
  
# 假设 ground_truth 是一个列表，包含了所有的真实标签  
# ground_truth = ['class1', 'class2', 'class1', 'class2', ...]  
  
# 创建一个列表，用于存储CSV的行  
csv_rows = []  
  
# 标题行  
header = ['Class', 'Precision', 'Recall', 'F1-score', 'Support']  
csv_rows.append(header)  
  
# 遍历 class_results 字典  
for class_label, metrics in class_results.items():  
    row = [class_label, "{:.2f}".format(metrics['precision']*100), "{:.2f}".format(metrics['recall']*100), "{:.2f}".format(metrics['f1-score']*100), metrics['support']]  
    csv_rows.append(row)  
  
# 添加 ground_truth 的统计信息  
ground_truth_counter = Counter(ground_truth)  
for class_label, count in ground_truth_counter.items():  
    row = [class_label, 'N/A', 'N/A', 'N/A', 'Support: ' + str(count)]  
    csv_rows.append(row)  
  
# 打开一个文件以写入CSV  
with open('./output/results1.csv', 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile)  
    # 写入所有行  
    writer.writerows(csv_rows)  
  
print("CSV file has been written.")

CSV file has been written.
